## 실제는 오타가 많다. 띄어쓰기 등
- 이모티콘을 많이 쓴다 ㅠㅠ, ^^
- 감성 분석에 편리하다. 묶어 줘야 한다
- Text cleaning: 노가다를 가능한 적게해야 한다
- 우선 순위가 낮은 것은 무시해야 한다.
- Mecap 가장 빠르다


### 크로스 밸리데이션 할 때
- 문제로 나누어진 형식대로 해야 한다
- 예를 들어 2014년 데이터로 2015를 예측하는 문제라면 모델도 그렇게 만들어야 한다
- 랜덤하게 나누는 것 주의

In [2]:
import pandas as pd

## 데이터셋 읽기

In [3]:
train = pd.read_csv('train.tsv', sep='\t', index_col='PhraseId')
print(train.shape)
train.head(3)


(156060, 3)


,SentenceId,Phrase,Sentiment
PhraseId,,,
1,1,A series of escapades demonstrating the adage ...,1
2,1,A series of escapades demonstrating the adage ...,2
3,1,A series,2


In [4]:
test = pd.read_csv('test.tsv', sep='\t', index_col='PhraseId')
print(test.shape)
test.head


(66292, 2)


<bound method NDFrame.head of           SentenceId                                             Phrase
PhraseId                                                               
156061          8545  An intermittently pleasing but mostly routine ...
156062          8545  An intermittently pleasing but mostly routine ...
156063          8545                                                 An
156064          8545  intermittently pleasing but mostly routine effort
156065          8545         intermittently pleasing but mostly routine
156066          8545                        intermittently pleasing but
156067          8545                            intermittently pleasing
156068          8545                                     intermittently
156069          8545                                           pleasing
156070          8545                                                but
156071          8545                                     mostly routine
156072          8545              

## 전처리

## clean text

In [5]:
def clean_text(word):
    word = word.replace("disappointments", "disappointment")
    word = word.replace("n't", "not")
    return word 

train["Phrase"] = train["Phrase"].apply(clean_text)


In [6]:
train[train["Phrase"].str.contains("n't")]

,SentenceId,Phrase,Sentiment
PhraseId,,,


In [32]:
!pip install nltk

In [33]:
from nltk.snowball import SnowBallStemmer

ModuleNotFoundError: No module named 'nltk.snowball'

 ### One Hot Encoder

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 50000, lowercase=False, ngram_range=(1,3))
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [8]:
vectorizer.fit(train["Phrase"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [9]:
X_train = vectorizer.transform(train["Phrase"])
print(X_train.shape)

X_train
columns = vectorizer.get_feature_names()
train_vector = pd.DataFrame(X_train.toarray(), columns= columns)

(156060, 50000)


In [10]:
columns

['000',
 '000 Leagues',
 '000 Leagues Under',
 '000 times',
 '10',
 '10 000',
 '10 000 times',
 '10 minutes',
 '10 minutes prior',
 '10 or',
 '10 or 15',
 '10 seconds',
 '10 set',
 '10 year',
 '10 year delay',
 '10 year old',
 '10 years',
 '10 years ago',
 '100',
 '100 minute',
 '100 minute running',
 '100 minutes',
 '100 years',
 '101',
 '101 premise',
 '101 study',
 '102',
 '102 minute',
 '102 minute infomercial',
 '103',
 '103 minute',
 '103 minute length',
 '104',
 '104 minutes',
 '105',
 '105 minutes',
 '105 minutes seem',
 '10th',
 '10th grade',
 '10th grade learning',
 '11',
 '11 period',
 '110',
 '12',
 '12 shot',
 '12 year',
 '12 year old',
 '127',
 '12th',
 '13',
 '13 Conversations',
 '13 months',
 '13 months and',
 '13 rating',
 '13 was',
 '13 was going',
 '13th',
 '13th by',
 '13th by way',
 '14',
 '14 year',
 '14 year old',
 '146',
 '146 minutes',
 '146 minutes of',
 '15',
 '15 minutes',
 '15 year',
 '15 years',
 '15 years old',
 '16',
 '163',
 '163 minutes',
 '170',
 '170

In [11]:
"reccomend" in columns

False

In [12]:
"disappoint" in columns

True

In [15]:
y_train = train["Sentiment"]

In [16]:
X_test = vectorizer.transform(test["Phrase"])
print(X_test.shape)


(66292, 50000)


In [17]:
from sklearn.linear_model import SGDClassifier
seed = 17
model = SGDClassifier(random_state=seed) # 랜덤 셔플링

In [18]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

predictions = cross_val_predict(model, X_train, y_train, cv = 5)
score = accuracy_score(y_train, predictions)
print(score)

0.573484557222


In [19]:
result = pd.DataFrame({'actual': y_train, 'predict': predictions})
result["distance"] = (result["actual"] - result["predict"]).abs()
result["Phrase"] = train["Phrase"]
result = result.sort_values("distance", ascending= False)
result.head(500).to_csv("Result.csv")
print(result.shape)
result.head()

(156060, 4)


,actual,predict,distance,Phrase
PhraseId,,,,
148792,4,0,4,"It is , however , a completely honest , open-h..."
124283,0,4,4,will find their humor-seeking dollars best spe...
140202,4,0,4,go down as one of the all-time great apocalyps...
140201,4,0,4,might go down as one of the all-time great apo...
70805,4,0,4,Maggie Smith as the Ya-Ya member with the O2-t...


In [20]:
result[result["Phrase"].str.contains("recommend")].head

<bound method NDFrame.head of           actual  predict  distance  \
PhraseId                              
36735          0        4         4   
80728          4        0         4   
36734          0        3         3   
80730          4        1         3   
80729          4        1         3   
80727          3        0         3   
100358         4        1         3   
100359         4        1         3   
37150          1        3         2   
37149          1        3         2   
139088         2        0         2   
89828          4        2         2   
36726          1        3         2   
36733          1        3         2   
58502          3        1         2   
124196         3        1         2   
96941          3        1         2   
96942          3        1         2   
100357         3        1         2   
22232          4        2         2   
22230          1        3         2   
22229          1        3         2   
22225          1        3         

In [21]:
result[result["Phrase"].str.contains("recommend")]["distance"].mean()

0.76884422110552764

In [22]:
result[result["Phrase"].str.contains("not funny")]["distance"].mean()

0.72727272727272729

In [23]:
model.fit (X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=17, shuffle=True, verbose=0,
       warm_start=False)

In [24]:
predictions = model.predict(X_test)

In [25]:
print(predictions.shape)
predictions[:30]

(66292,)


array([3, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 3,
       2, 2, 2, 2, 2, 3, 2])

In [26]:
submission = pd.read_csv("sampleSubmission.csv", index_col = "PhraseId")
# submission["Sentiment"] = predictions

In [27]:
submission["Sentiment"] = predictions

In [28]:
submission.to_csv("first.csv")